In [ ]:
%load_ext autoreload
%autoreload 2

from incomer.data import DataManager
import pandas as pd

filename = '../data/EURUSD60.csv'
cycles = [5, 20, 50, 100, 200, 600, 1000]
num_bars = 300
forward_bars = 48
lookbacks = 20
valid_split = 0.8

dm = DataManager(filename, cycles, num_bars=num_bars)

xtrain, ytrain, xtest, ytest = dm.create_train_data(valid_split, lookbacks, forward_bars)

print(xtrain.shape)
print(xtest.shape)

In [ ]:
len(dm.op)

In [ ]:
from incomer.models.transformer import TradeTransformer

trader = TradeTransformer()
trader.build_model(
    input_shape=xtrain.shape[1:],
    head_size=128,
    num_heads=16,
    ff_dim=4,
    num_transformer_blocks=16,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.3,
)
# trader.model.summary()

In [ ]:
import numpy as np

def replace_nan(x, value=-1):
    nan = np.isnan(x)
    x[nan] = -1
    return x

xtrain = replace_nan(xtrain, value=-1)
xtest = replace_nan(xtest, value=-1)
dm.body = replace_nan(dm.body, value=0)

In [ ]:
trader.train(
    xtrain,
    ytrain,
    xtest,
    dm.body,
    dm.op,
    epochs=999999999,
    batch_size=16,
    valid_splits=0.8,
    save_path="data/eurusd_h1/best",
)
